In [137]:
import pandas as pd

In [138]:
# ----------------------------------------------------
# Step 1: 모든 데이터 파일 경로 정의
# ----------------------------------------------------
# Raw (원본) 데이터
path_sales = r'C:\hanchon\data\raw\sample_data_1016.csv'
path_store_index = r'C:\hanchon\data\raw\매장지수.xlsx'
path_market_index = r'C:\hanchon\data\raw\상권지수.xlsx'

# Processed (가공) 데이터
path_weather = r'C:\hanchon\data\processed\weather_processed_20230101-20250930.csv'
path_population = r'C:\hanchon\data\processed\population_samsung_quarterly.csv'
path_holidays = r'C:\hanchon\data\processed\holidays_20230101-20250930.csv'

# ----------------------------------------------------
# Step 2: 모든 데이터 불러오기
# ----------------------------------------------------
print("1. 데이터 불러오는 중...")
df_sales = pd.read_csv(path_sales)
df_store = pd.read_excel(path_store_index)
df_market = pd.read_excel(path_market_index)

df_weather = pd.read_csv(path_weather)
df_population = pd.read_csv(path_population)
df_holidays = pd.read_csv(path_holidays)

print("데이터 불러오기 완료")

1. 데이터 불러오는 중...
데이터 불러오기 완료


In [139]:
df_weather.isnull().sum()

date                 0
temp_avg             0
temp_min             0
temp_max             0
precipitation_sum    0
dtype: int64

In [140]:
# pip install openpyxl

In [141]:
print("--- 1&2. 데이터 로드 확인 ---")
print(f"Sales (판매): {df_sales.shape}")
print(f"Store Index (매장지수): {df_store.shape}")
print(f"Market Index (상권지수): {df_market.shape}")
print(f"Weather (날씨): {df_weather.shape}")
print(f"Population (인구): {df_population.shape}")
print(f"Holidays (공휴일): {df_holidays.shape}")

--- 1&2. 데이터 로드 확인 ---
Sales (판매): (2433346, 8)
Store Index (매장지수): (528, 4)
Market Index (상권지수): (60, 4)
Weather (날씨): (1003, 5)
Population (인구): (10, 5)
Holidays (공휴일): (1004, 3)


In [142]:
# ----------------------------------------------------
# Step 3: 기준 데이터(df_sales) 정리 및 날짜 기반 데이터 병합
# ----------------------------------------------------
print("2. 날짜 기반 데이터 병합 (날씨, 공휴일)...")
df_main = df_sales.copy()
df_main.rename(columns={'일자': 'date'}, inplace=True)
df_main= df_main[df_main['date'] <= '2025-09-30'].copy()

df_main['date'] = pd.to_datetime(df_main['date'])
df_weather['date'] = pd.to_datetime(df_weather['date'])
df_holidays['date'] = pd.to_datetime(df_holidays['date'])

# 3. '일자'를 기준으로 날씨와 공휴일 데이터를 병합(merge)
#    컬럼명 '일자'를 'date'로 바꿔서 통일합니다.

df_main = pd.merge(df_main, df_weather, on='date', how='left')
df_main = pd.merge(df_main, df_holidays, on='date', how='left')

print("날짜 기반 데이터 병합 (날씨, 공휴일) 완료")

2. 날짜 기반 데이터 병합 (날씨, 공휴일)...
날짜 기반 데이터 병합 (날씨, 공휴일) 완료


In [143]:
# ----------------------------------------------------
# Step 4: 시간 기준 컬럼 생성 (병합을 위한 Key)
# ----------------------------------------------------
print("3. 시간 기준 컬럼 생성 (year, month, quarter)...")
df_main['year'] = df_main['date'].dt.year
df_main['month'] = df_main['date'].dt.month
df_main['quarter'] = df_main['date'].dt.quarter

print("시간 기준 컬럼 생성 (year, month, quarter) 완료")

3. 시간 기준 컬럼 생성 (year, month, quarter)...
시간 기준 컬럼 생성 (year, month, quarter) 완료


In [144]:
print("--- 3. 시간 기준 컬럼 생성 확인 ---")
print(df_main[['date', 'year', 'month', 'quarter']].head())

--- 3. 시간 기준 컬럼 생성 확인 ---
        date  year  month  quarter
0 2023-01-01  2023      1        1
1 2023-01-01  2023      1        1
2 2023-01-01  2023      1        1
3 2023-01-01  2023      1        1
4 2023-01-01  2023      1        1


In [145]:
# ----------------------------------------------------
# Step 5: 월/분기 기반 데이터 병합 (지수, 인구)
# ----------------------------------------------------
print("4. 월/분기 기반 데이터 병합 (지수, 인구)...")

# --- 5b. [월/분기 기준] 병합 (지수, 인구) ---
# 상권지수 (Key: year, month, 상권)
df_market.rename(columns={'연도': 'year', '월': 'month'}, inplace=True)
df_main = pd.merge(df_main, df_market, on=['year', 'month', '상권'], how='left')

# 매장지수 (Key: month, 상권, 매장명)
# '매장코드'를 '매장명'으로 통일 (EDA로 확인했다는 논리)
df_store.rename(columns={'월': 'month', '매장코드': '매장명'}, inplace=True) 
df_main = pd.merge(df_main, df_store, on=['month', '상권', '매장명'], how='left')

# 인구 (Key: year, quarter)
df_main = pd.merge(df_main, df_population, on=['year', 'quarter'], how='left')

4. 월/분기 기반 데이터 병합 (지수, 인구)...


In [146]:
df_main.head()

,date,상권,홀배달여부,매장명,메뉴코드,메뉴명,매출액,수량,temp_avg,temp_min,...,is_holiday,is_event,year,month,quarter,상권지수,매장지수,floating_pop,resident_pop,working_pop
0,2023-01-01,오피스,홀,Store_001,W11010101,설렁탕,191657.0,17,-0.2,-4.3,...,1,0,2023,1,1,NaN,NaN,7115705.0,25864.0,886.0
1,2023-01-01,오피스,홀,Store_001,W11010101,설렁탕(파빼고),18298.0,1,-0.2,-4.3,...,1,0,2023,1,1,NaN,NaN,7115705.0,25864.0,886.0
2,2023-01-01,오피스,홀,Store_001,W11010101,설렁탕(밥따로),75905.0,8,-0.2,-4.3,...,1,0,2023,1,1,NaN,NaN,7115705.0,25864.0,886.0
3,2023-01-01,오피스,홀,Store_001,W11010202,특양지설렁탕,36348.0,2,-0.2,-4.3,...,1,0,2023,1,1,NaN,NaN,7115705.0,25864.0,886.0
4,2023-01-01,오피스,홀,Store_001,W11010301,어린이설렁탕,22021.0,4,-0.2,-4.3,...,1,0,2023,1,1,NaN,NaN,7115705.0,25864.0,886.0


In [147]:
print("--- 4. 데이터 병합 확인 (NaN은 정상) ---")
# 새로 병합된 컬럼들의 결측치(NaN) 개수를 확인
check_cols = [
    'temp_avg',        # 날씨
    'temp_min',
    'temp_max',
    'precipitation_sum',
    'is_holiday',      # 공휴일
    '상권지수',          # 상권지수
    '매장지수',          # 매장지수
    'floating_pop',     # 인구
    'resident_pop',
    'working_pop'

]
print(df_main[check_cols].isnull().sum())

--- 4. 데이터 병합 확인 (NaN은 정상) ---
temp_avg                2231
temp_min                2231
temp_max                2231
precipitation_sum       2231
is_holiday                 0
상권지수                 1748816
매장지수                  411045
floating_pop          229157
resident_pop          229157
working_pop           229157
dtype: int64


In [148]:
df_main.tail()

,date,상권,홀배달여부,매장명,메뉴코드,메뉴명,매출액,수량,temp_avg,temp_min,...,is_holiday,is_event,year,month,quarter,상권지수,매장지수,floating_pop,resident_pop,working_pop
2398645,2025-09-30,교외,홀,Store_050,W11102601,참이슬후레쉬,125083.0,19,19.731319,17.3,...,0,0,2025,9,3,1.045381,1.1946,NaN,NaN,NaN
2398646,2025-09-30,교외,홀,Store_050,W11110101,콜라,4041.0,3,19.731319,17.3,...,0,0,2025,9,3,1.045381,1.1946,NaN,NaN,NaN
2398647,2025-09-30,교외,배달,Store_050,W11110101,배)콜라,1025.0,1,19.731319,17.3,...,0,0,2025,9,3,1.045381,1.1946,NaN,NaN,NaN
2398648,2025-09-30,교외,배달,Store_050,W11139902,배달팁 500,4155.0,28,19.731319,17.3,...,0,0,2025,9,3,1.045381,1.1946,NaN,NaN,NaN
2398649,2025-09-30,교외,배달,Store_050,W11139902,배달팁 1000,26752.0,19,19.731319,17.3,...,0,0,2025,9,3,1.045381,1.1946,NaN,NaN,NaN


In [ ]:
# ----------------------------------------------------
# Step 6: 최종 데이터 확인, 결측치 처리, 저장
# ----------------------------------------------------
print("5. 최종 데이터 확인, 결측치 처리, 저장...")

final_dataset = df_main.copy()
print(f"결측치 처리 시작 전. Shape: {final_dataset.shape}")

# ----------------------------------------------------
# 처리 1: 날씨 결측치 (temp_avg 등 4종)
# ----------------------------------------------------
print(f"\n--- 1. 날씨 결측치 처리 ---")
print(f"처리 전 NaNs (temp_avg): {final_dataset['temp_avg'].isnull().sum()}")
weather_cols = ['temp_avg', 'temp_min', 'temp_max', 'precipitation_sum']
# 논리: 날씨는 연속적이므로 선형 보간법으로 채움
final_dataset[weather_cols] = final_dataset[weather_cols].interpolate(method='linear')
# 맨 앞/뒤가 비었을 경우를 대비해 bfill/ffill 추가
final_dataset[weather_cols] = final_dataset[weather_cols].fillna(method='bfill').fillna(method='ffill')
print(f"처리 후 NaNs (temp_avg): {final_dataset['temp_avg'].isnull().sum()}")

# ----------------------------------------------------
# 처리 2: 상권지수 결측치
# ----------------------------------------------------
print(f"\n--- 2. 상권지수 결측치 처리 ---")
print(f"처리 전 NaNs: {final_dataset['상권지수'].isnull().sum()}")
# 논리: 최신(2025년) 데이터로 과거(23-24년) 보정 + 안전망
# 2a. '상권', '월', '연도' 순으로 정렬
final_dataset.sort_values(by=['상권', 'month', 'year'], inplace=True)
# 2b. '상권'과 '월'을 기준으로 bfill (최신 데이터로 과거 채우기)
final_dataset['상권지수'] = final_dataset.groupby(['상권', 'month'])['상권지수'].transform(
    lambda x: x.bfill().ffill()
)
# 2c. 1차 안전망: '월' 전체의 평균으로 남은 NaN 채우기
month_avg_market = final_dataset.groupby('month')['상권지수'].transform('mean')
final_dataset['상권지수'] = final_dataset['상권지수'].fillna(month_avg_market)
# 2d. 2차 안전망: '전체' 평균으로 최종 NaN 채우기
final_dataset['상권지수'] = final_dataset['상권지수'].fillna(final_dataset['상권지수'].mean())
print(f"처리 후 NaNs: {final_dataset['상권지수'].isnull().sum()}")

# ----------------------------------------------------
# 처리 3: 매장지수 결측치
# ----------------------------------------------------
print(f"\n--- 3. 매장지수 결측치 처리 ---")
print(f"처리 전 NaNs: {final_dataset['매장지수'].isnull().sum()}")
# 논리: '같은 상권, 같은 월'의 평균으로 추론 + 안전망
# 3a. [상권, 월]별 평균 계산
market_month_avg_index = df_store.groupby(['상권', 'month'])['매장지수'].mean().reset_index()
market_month_avg_index.rename(columns={'매장지수': '상권월평균_매장지수'}, inplace=True)
# 3b. 평균값 병합
final_dataset = pd.merge(final_dataset, market_month_avg_index, on=['상권', 'month'], how='left')
# 3c. NaN을 평균값으로 채우기
final_dataset['매장지수'] = final_dataset['매장지수'].fillna(final_dataset['상권월평균_매장지수'])
# 3d. 1차 안전망: '월' 전체 평균
month_avg_store = final_dataset.groupby('month')['매장지수'].transform('mean')
final_dataset['매장지수'] = final_dataset['매장지수'].fillna(month_avg_store)
# 3e. 2차 안전망: '전체' 평균
final_dataset['매장지수'] = final_dataset['매장지수'].fillna(final_dataset['매장지수'].mean())
# 3f. 임시 컬럼 제거
final_dataset.drop(columns=['상권월평균_매장지수'], errors='ignore', inplace=True)
print(f"처리 후 NaNs: {final_dataset['매장지수'].isnull().sum()}")


# ----------------------------------------------------
# 처리 4: 인구 결측치 (floating_pop 등 3종)
# ----------------------------------------------------
print(f"\n--- 4. 인구 결측치 처리 ---")
print(f"처리 전 NaNs (floating_pop): {final_dataset['floating_pop'].isnull().sum()}")
# 논리: 최신(Q2) 값으로 누락된 Q3를 채움 (ffill)
# ffill을 위해 날짜순 정렬
final_dataset.sort_values(by='date', inplace=True) 
pop_cols = ['floating_pop', 'resident_pop', 'working_pop']
final_dataset[pop_cols] = final_dataset[pop_cols].fillna(method='ffill')
print(f"처리 후 NaNs (floating_pop): {final_dataset['floating_pop'].isnull().sum()}")


# ----------------------------------------------------
# 처리 5: 메뉴 결측치 및 '단가' 피처 생성
# ----------------------------------------------------
import numpy as np

print(f"\n--- 5. 메뉴 결측치 처리 및 단가 피처 생성 ---")
print(f"처리 전 NaNs (메뉴명): {final_dataset['메뉴명'].isnull().sum()}")

# 5a. 메뉴 결측치는 'UNKNOWN' 카테고리로 지정
final_dataset['메뉴코드'] = final_dataset['메뉴코드'].fillna('UNKNOWN')
final_dataset['메뉴명'] = final_dataset['메뉴명'].fillna('UNKNOWN')

print(f"처리 후 NaNs (메뉴명): {final_dataset['메뉴명'].isnull().sum()}")

# 5b. '단가(unit_price)' 피처 생성
#     수량이 0일 때 0으로 나누는 오류(ZeroDivisionError)를 방지
final_dataset['unit_price'] = np.where(
    final_dataset['수량'] > 0, # 조건: 수량이 0보다 크면
    final_dataset['매출액'] / final_dataset['수량'], # True: 매출액 / 수량
    0  # False: 0으로 처리 (또는 NaN으로 처리 후 보간)
)

print("'unit_price' (단가) 피처 생성 완료")
print(final_dataset[['매출액', '수량', 'unit_price']].head())

# ----------------------------------------------------
# 최종 정리 및 저장
# ----------------------------------------------------
# 6. 최종 정리: 날짜순 정렬, 중복 컬럼 제거
final_dataset.sort_values(by='date', inplace=True)
final_dataset = final_dataset.loc[:, ~final_dataset.columns.duplicated()]

print("\n--- ★★★ 최종 결측치 현황 ★★★ ---")
print(final_dataset.isnull().sum())

# 7. 최종 저장
output_path = r'C:\hanchon\data\processed\final_integrated_dataset.csv'
final_dataset.to_csv(output_path, index=False)

print(f"\n✅ 모든 데이터 통합 및 전처리 완료! 경로: {output_path}")
print("최종 데이터셋 Shape:", final_dataset.shape)
final_dataset.info()

5. 최종 데이터 확인, 결측치 처리, 저장...
결측치 처리 시작 전. Shape: (2398650, 22)


In [ ]:
# # ----------------------------------------------------
# # 처리 1: 날씨 결측치 (temp_avg 등 4종)
# # ----------------------------------------------------
# print(f"\n--- 1. 날씨 결측치 처리 ---")
# print(f"처리 전 NaNs (temp_avg): {final_dataset['temp_avg'].isnull().sum()}")
# weather_cols = ['temp_avg', 'temp_min', 'temp_max', 'precipitation_sum']
# # 논리: 날씨는 연속적이므로 선형 보간법으로 채움
# final_dataset[weather_cols] = final_dataset[weather_cols].interpolate(method='linear')
# # 맨 앞/뒤가 비었을 경우를 대비해 bfill/ffill 추가
# final_dataset[weather_cols] = final_dataset[weather_cols].fillna(method='bfill').fillna(method='ffill')
# print(f"처리 후 NaNs (temp_avg): {final_dataset['temp_avg'].isnull().sum()}")


--- 1. 날씨 결측치 처리 ---
처리 전 NaNs (temp_avg): 2231
처리 후 NaNs (temp_avg): 0


C:\Users\leesk\AppData\Local\Temp\ipykernel_73412\2638182704.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_dataset[weather_cols] = final_dataset[weather_cols].fillna(method='bfill').fillna(method='ffill')


In [ ]:
# # ----------------------------------------------------
# # 처리 2: 상권지수 결측치
# # ----------------------------------------------------
# print(f"\n--- 2. 상권지수 결측치 처리 ---")
# print(f"처리 전 NaNs: {final_dataset['상권지수'].isnull().sum()}")
# # 논리: 최신(2025년) 데이터로 과거(23-24년) 보정 + 안전망
# # 2a. '상권', '월', '연도' 순으로 정렬
# final_dataset.sort_values(by=['상권', 'month', 'year'], inplace=True)
# # 2b. '상권'과 '월'을 기준으로 bfill (최신 데이터로 과거 채우기)
# final_dataset['상권지수'] = final_dataset.groupby(['상권', 'month'])['상권지수'].transform(
#     lambda x: x.bfill().ffill()
# )
# # 2c. 1차 안전망: '월' 전체의 평균으로 남은 NaN 채우기
# month_avg_market = final_dataset.groupby('month')['상권지수'].transform('mean')
# final_dataset['상권지수'] = final_dataset['상권지수'].fillna(month_avg_market)
# # 2d. 2차 안전망: '전체' 평균으로 최종 NaN 채우기
# final_dataset['상권지수'] = final_dataset['상권지수'].fillna(final_dataset['상권지수'].mean())
# print(f"처리 후 NaNs: {final_dataset['상권지수'].isnull().sum()}")


--- 2. 상권지수 결측치 처리 ---
처리 전 NaNs: 1748816
처리 후 NaNs: 0


In [ ]:
# # ----------------------------------------------------
# # 처리 3: 매장지수 결측치
# # ----------------------------------------------------
# print(f"\n--- 3. 매장지수 결측치 처리 ---")
# print(f"처리 전 NaNs: {final_dataset['매장지수'].isnull().sum()}")
# # 논리: '같은 상권, 같은 월'의 평균으로 추론 + 안전망
# # 3a. [상권, 월]별 평균 계산
# market_month_avg_index = df_store.groupby(['상권', 'month'])['매장지수'].mean().reset_index()
# market_month_avg_index.rename(columns={'매장지수': '상권월평균_매장지수'}, inplace=True)
# # 3b. 평균값 병합
# final_dataset = pd.merge(final_dataset, market_month_avg_index, on=['상권', 'month'], how='left')
# # 3c. NaN을 평균값으로 채우기
# final_dataset['매장지수'] = final_dataset['매장지수'].fillna(final_dataset['상권월평균_매장지수'])
# # 3d. 1차 안전망: '월' 전체 평균
# month_avg_store = final_dataset.groupby('month')['매장지수'].transform('mean')
# final_dataset['매장지수'] = final_dataset['매장지수'].fillna(month_avg_store)
# # 3e. 2차 안전망: '전체' 평균
# final_dataset['매장지수'] = final_dataset['매장지수'].fillna(final_dataset['매장지수'].mean())
# # 3f. 임시 컬럼 제거
# final_dataset.drop(columns=['상권월평균_매장지수'], errors='ignore', inplace=True)
# print(f"처리 후 NaNs: {final_dataset['매장지수'].isnull().sum()}")




--- 3. 매장지수 결측치 처리 ---
처리 전 NaNs: 411045
처리 후 NaNs: 0


In [ ]:
# # ----------------------------------------------------
# # 처리 4: 인구 결측치 (floating_pop 등 3종)
# # ----------------------------------------------------
# print(f"\n--- 4. 인구 결측치 처리 ---")
# print(f"처리 전 NaNs (floating_pop): {final_dataset['floating_pop'].isnull().sum()}")
# # 논리: 최신(Q2) 값으로 누락된 Q3를 채움 (ffill)
# # ffill을 위해 날짜순 정렬
# final_dataset.sort_values(by='date', inplace=True) 
# pop_cols = ['floating_pop', 'resident_pop', 'working_pop']
# final_dataset[pop_cols] = final_dataset[pop_cols].fillna(method='ffill')
# print(f"처리 후 NaNs (floating_pop): {final_dataset['floating_pop'].isnull().sum()}")




--- 4. 인구 결측치 처리 ---
처리 전 NaNs (floating_pop): 229157
처리 후 NaNs (floating_pop): 0


C:\Users\leesk\AppData\Local\Temp\ipykernel_73412\2787115110.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_dataset[pop_cols] = final_dataset[pop_cols].fillna(method='ffill')


In [ ]:
# # ----------------------------------------------------
# # 처리 5: 메뉴 결측치 및 '단가' 피처 생성
# # ----------------------------------------------------
# import numpy as np

# print(f"\n--- 5. 메뉴 결측치 처리 및 단가 피처 생성 ---")
# print(f"처리 전 NaNs (메뉴명): {final_dataset['메뉴명'].isnull().sum()}")

# # 5a. 메뉴 결측치는 'UNKNOWN' 카테고리로 지정
# final_dataset['메뉴코드'] = final_dataset['메뉴코드'].fillna('UNKNOWN')
# final_dataset['메뉴명'] = final_dataset['메뉴명'].fillna('UNKNOWN')

# print(f"처리 후 NaNs (메뉴명): {final_dataset['메뉴명'].isnull().sum()}")

# # 5b. '단가(unit_price)' 피처 생성
# #     수량이 0일 때 0으로 나누는 오류(ZeroDivisionError)를 방지
# final_dataset['unit_price'] = np.where(
#     final_dataset['수량'] > 0, # 조건: 수량이 0보다 크면
#     final_dataset['매출액'] / final_dataset['수량'], # True: 매출액 / 수량
#     0  # False: 0으로 처리 (또는 NaN으로 처리 후 보간)
# )

# print("'unit_price' (단가) 피처 생성 완료")
# print(final_dataset[['매출액', '수량', 'unit_price']].head())


--- 5. 메뉴 결측치 처리 및 단가 피처 생성 ---
처리 전 NaNs (메뉴명): 43694
처리 후 NaNs (메뉴명): 0
'unit_price' (단가) 피처 생성 완료
              매출액   수량    unit_price
0       1397204.0   92  15187.000000
956744   252443.0   19  13286.473684
956745   663426.0   96   6910.687500
956746   154183.0   13  11860.230769
956747  1980350.0  252   7858.531746


In [ ]:
# # ----------------------------------------------------
# # 최종 정리 및 저장
# # ----------------------------------------------------
# # 6. 최종 정리: 날짜순 정렬, 중복 컬럼 제거
# final_dataset.sort_values(by='date', inplace=True)
# final_dataset = final_dataset.loc[:, ~final_dataset.columns.duplicated()]

# print("\n--- ★★★ 최종 결측치 현황 ★★★ ---")
# print(final_dataset.isnull().sum())

# # 7. 최종 저장
# output_path = r'C:\hanchon\data\processed\final_integrated_dataset.csv'
# final_dataset.to_csv(output_path, index=False)

# print(f"\n✅ 모든 데이터 통합 및 전처리 완료! 경로: {output_path}")
# print("최종 데이터셋 Shape:", final_dataset.shape)
# final_dataset.info()


--- ★★★ 최종 결측치 현황 ★★★ ---
date                 0
상권                   0
홀배달여부                0
매장명                  0
메뉴코드                 0
메뉴명                  0
매출액                  0
수량                   0
temp_avg             0
temp_min             0
temp_max             0
precipitation_sum    0
is_holiday           0
is_event             0
year                 0
month                0
quarter              0
상권지수                 0
매장지수                 0
floating_pop         0
resident_pop         0
working_pop          0
unit_price           0
dtype: int64

✅ 모든 데이터 통합 및 전처리 완료! 경로: C:\hanchon\data\processed\final_integrated_dataset.csv
최종 데이터셋 Shape: (2398650, 23)
<class 'pandas.core.frame.DataFrame'>
Index: 2398650 entries, 0 to 2078897
Data columns (total 23 columns):
 #   Column             Dtype         
---  ------             -----         
 0   date               datetime64[ns]
 1   상권                 object        
 2   홀배달여부              object        
 3   매장명         

In [157]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2398650 entries, 0 to 2078897
Data columns (total 23 columns):
 #   Column             Dtype         
---  ------             -----         
 0   date               datetime64[ns]
 1   상권                 object        
 2   홀배달여부              object        
 3   매장명                object        
 4   메뉴코드               object        
 5   메뉴명                object        
 6   매출액                float64       
 7   수량                 int64         
 8   temp_avg           float64       
 9   temp_min           float64       
 10  temp_max           float64       
 11  precipitation_sum  float64       
 12  is_holiday         int64         
 13  is_event           int64         
 14  year               int32         
 15  month              int32         
 16  quarter            int32         
 17  상권지수               float64       
 18  매장지수               float64       
 19  floating_pop       float64       
 20  resident_pop       float64   